In [1]:
import wikipedia

In [ ]:
# Fetch wikipedia page from a page name. Tries with and without autosuggest. Return None if no page was found.
def get_wiki_page(link):
    try:
        composer = wikipedia.page(title=link, auto_suggest=False)
        return composer
    except:
        pass
    try:
        composer = wikipedia.page(title=link, auto_suggest=True)
        return composer
    except:
        return None

# Bulk addition of a tag to the information dict.
def add_information(info_dict, name_set, tag_key, tag_value):
    for name in name_set:
        if name in info_dict:
            info_dict[name][tag_key] = tag_value
        else:
            info_dict[name] = {}
            info_dict[name][tag_key] = tag_value

# composer -> information dict (= birthplace -> Pittsburgh ...)
composer_information = {}
# Set of wikipedia links to composer pages
links_set = set()

composers = set(get_wiki_page("List_of_medieval_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'medieval')

composers = set(get_wiki_page("List_of_Baroque_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'baroque')

composers = set(get_wiki_page("List_of_Classical-era_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'classical')

composers = set(get_wiki_page("List_of_Romantic-era_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'romantic')

composers = set(get_wiki_page("List_of_20th-century_classical_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'modern')

composers = set(get_wiki_page("List_of_21st-century_classical_composers").links)
links_set |= composers
add_information(composer_information, composers, 'genre', 'contemporary')

links = list(links_set)
relations = {}
composer_count = 0
for link in links:
    try:
        # Obtain wikipedia page corresponding to the composer
        composer = get_wiki_page(link)
        if composer == None:
            continue

        # Check if the page indeed relates to a composer
        is_composer = False
        for category in composer.categories:
            if 'composer' in category.lower():
                is_composer = True
        if not is_composer:
            continue

        # Only keep composers for which a birthplace and date of birth was provided
        has_birthplace = False
        has_dob = False
        for c in composer.categories:
            if c.startswith('People from '):
                composer_information[link]['birthplace'] = c[len('People from '):]
                has_birthplace = True
            if c.endswith(' births'):
                composer_information[link]['dob'] = c[:-len(' births')]
                has_dob = True
        if not (has_birthplace and has_dob):
            del composer_information[link]
            continue

        # If all the filters are passed, add to relations list
        relations[link] = composer.links
        composer_count += 1
        if (composer_count == 1 or composer_count % 10 == 0):
            print("composer_count: " + str(composer_count))
            
    except KeyboardInterrupt:
        print("Exiting. [KeyboardInterrupt]")
    except :
        print("ERROR " + link + " -----------------------------")
        
# In the end we only consider the relations of composers with enough data available        
allComposers = composer_information.keys()


composer_count: 10
composer_count: 20
ERROR Jeanne Leleu------------------------------
composer_count: 30
composer_count: 40
composer_count: 50
ERROR Paul I, Prince Esterházy------------------------------
ERROR Julie Reisserová------------------------------
ERROR Michel van der Aa------------------------------
ERROR Hieronymus Praetorius------------------------------
composer_count: 60
composer_count: 70
ERROR Johannes Cuvelier------------------------------
composer_count: 80


In [ ]:
def filterComposerRelationsOnly(relationsOfComposer, allComposers):
    return [relation for relation in relationsOfComposer if relation in allComposers]

relations = {k: filterComposerRelationsOnly(relationsOfComposer, allComposers) for k, relationsOfComposer in relations.items()}

In [ ]:
file = open("followerFolloweeRomantic.txt","w") 
for followeeComposer,followedComposers in relations.items():
    for followedComposer in followedComposers:
        file.write(followeeComposer + "," + followedComposer + "\n")  
file.close() 

In [ ]:
file = open('composer_information.txt','w')
for name, info in composer_information.items():
    if len(info) == 3:
        file.write(name + '\t' + info['dob'] + '\t' + info['birthplace'] + '\t' + info['genre'] + '\n')
file.close()